In [19]:
import os
import tweepy as tw
import pandas as pd
import csv
import pandas as pd
import glob
from nltk.corpus import stopwords
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from sklearn.model_selection import train_test_split
import keras.models
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras import backend as K
from sklearn.metrics import accuracy_score
from keras.models import load_model
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers.merge import concatenate
from numpy import argmax

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
extension = 'csv'
all_filenames = [i for i in glob.glob('/content/drive/MyDrive/twit/train/*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "aggregate.csv", index=False, encoding='utf-8-sig')

In [21]:
train = pd.read_csv('/content/drive/MyDrive/twit/train/aggregate.csv')

train.head()

,Column1,Column2,Column3
0,6.289490e+17,negative,dear @Microsoft the newOoffice for Mac is grea...
1,6.289770e+17,negative,@Microsoft how about you make a system that do...
2,6.290230e+17,negative,I may be ignorant on this issue but... should ...
3,6.291790e+17,negative,"Thanks to @microsoft, I just may be switching ..."
4,6.291860e+17,neutral,If I make a game as a #windows10 Universal App...


In [22]:
def remove_pattern(input_txt, pattern):
  r = re.findall(pattern, input_txt)
  for i in r:
    input_txt = re.sub(i, '', input_txt)        
  return input_txt

def clean_tweets(frame, column_name, remove_stop_words=True, lower_case=True, remove_special=True):
  frame = frame.drop_duplicates().reset_index(drop=True) #remove duplicate rows
  frame['Tweet_Clean_Text'] = np.vectorize(remove_pattern)(frame[column_name], "RT @[\w]*:") #remove twitter return handle
  frame.Tweet_Clean_Text = np.vectorize(remove_pattern)(frame['Tweet_Clean_Text'], "@[\w]*") #remove twitter handle
  frame.Tweet_Clean_Text = np.vectorize(remove_pattern)(frame['Tweet_Clean_Text'], "https?://[A-Za-z0-9./]*") #remove URLs

  if remove_special:
    frame.Tweet_Clean_Text = frame.Tweet_Clean_Text.str.replace("[^a-zA-Z#]", " ") #remove special characters except for #
  frame.Tweet_Clean_Text = frame.Tweet_Clean_Text.replace('\s+', ' ', regex=True) #remove extra spaces in between words
  if lower_case:
    frame.Tweet_Clean_Text = frame.Tweet_Clean_Text.apply(lambda x: x.lower())
  if remove_stop_words:
    frame.Tweet_Clean_Text = frame.Tweet_Clean_Text.apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')])) #remove stop words

  return frame

In [23]:
train_set = clean_tweets(train, 'Column3', remove_stop_words=False, remove_special=False)
train_set.head()

,Column1,Column2,Column3,Tweet_Clean_Text
0,6.289490e+17,negative,dear @Microsoft the newOoffice for Mac is grea...,"dear the newooffice for mac is great and all, ..."
1,6.289770e+17,negative,@Microsoft how about you make a system that do...,how about you make a system that doesn't eat ...
2,6.290230e+17,negative,I may be ignorant on this issue but... should ...,i may be ignorant on this issue but... should ...
3,6.291790e+17,negative,"Thanks to @microsoft, I just may be switching ...","thanks to , i just may be switching over to ."
4,6.291860e+17,neutral,If I make a game as a #windows10 Universal App...,if i make a game as a #windows10 universal app...


In [24]:
train_set["Sentiment_Value"] = train_set["Column2"].map({"neutral": 0, "positive": 1, "negative": 2})
label = to_categorical(train_set["Sentiment_Value"], 3)
train_set

,Column1,Column2,Column3,Tweet_Clean_Text,Sentiment_Value
0,6.289490e+17,negative,dear @Microsoft the newOoffice for Mac is grea...,"dear the newooffice for mac is great and all, ...",2
1,6.289770e+17,negative,@Microsoft how about you make a system that do...,how about you make a system that doesn't eat ...,2
2,6.290230e+17,negative,I may be ignorant on this issue but... should ...,i may be ignorant on this issue but... should ...,2
3,6.291790e+17,negative,"Thanks to @microsoft, I just may be switching ...","thanks to , i just may be switching over to .",2
4,6.291860e+17,neutral,If I make a game as a #windows10 Universal App...,if i make a game as a #windows10 universal app...,0
...,...,...,...,...,...
114213,2.103780e+17,neutral,It's a Wednesday girls night out as '90's band...,it's a wednesday girls night out as '90's band...,0
114214,2.451780e+17,positive,"night college course sorted, just have to enro...","night college course sorted, just have to enro...",1
114215,2.592810e+17,positive,For the 1st time in 30 years. For your splendi...,for the 1st time in 30 years. for your splendi...,1
114216,2.011140e+17,positive,NURSES DAY - 12 MAY 2012. Nursing: The heart b...,nurses day - 12 may 2012. nursing: the heart b...,1


In [25]:
train['l'] = train_set['Tweet_Clean_Text'].apply(lambda x: len(str(x).split(' ')))
print("mean length of sentence: " + str(train.l.mean()))
print("max length of sentence: " + str(train.l.max()))
print("std dev length of sentence: " + str(train.l.std()))

mean length of sentence: 16.71109632457231
max length of sentence: 40.0
std dev length of sentence: 6.599111305083614


In [26]:
sequence_length = train.l.max() #using the maximum length 

In [27]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_set['Tweet_Clean_Text'].values)

X = tokenizer.texts_to_sequences(train_set['Tweet_Clean_Text'].values)
X = pad_sequences(X, maxlen=int(sequence_length), padding='post')
X

print("training size " + str(len(x_train)))

voc_size = len(tokenizer.word_index) + 1
print("Vocab size: ", voc_size)
print("Input shape: ", X.shape)
print("Y_shape: " , label.shape)

training size 102796
Vocab size:  73456
Input shape:  (114218, 40)
Y_shape:  (114218, 3)


In [28]:
#FINAL DATASETS (Please note case sensitivity)- TRAINING: (X_train,Y_train)  VALIDATION: (X_val, Y_val)  TEST: (X_test, Y_test)

x_train, X_test, y_train, Y_test = train_test_split(X, label, test_size=0.10, shuffle=False, random_state=10)

X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.15, shuffle=True, random_state=10)

In [32]:
def recall_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    recall_score.__name__ = 'recall'
    return recall

def precision_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    precision_score.__name__ = 'precision'
    return precision


def f1_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1_metrics.__name__ = 'f1_metrics'
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

metrics = [
           keras.metrics.CategoricalAccuracy(),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall'),
           f1_metrics,
]

In [38]:
# stacked generalization with neural net meta model on blobs dataset

# load models from file
def load_all_models(model_names):
	all_models = list()
	for model_name in model_names:
		# define filename for this ensemble
		filename = '/content/drive/MyDrive/twit/train/models/' + model_name
		# load model from file
		model = load_model(filename,custom_objects={'f1_metrics':f1_metrics,'precision':precision_score,'recall':recall_score})
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models

# load all models
members = ['LSTM200.h5', 'LSTM150.h5', 'LSTM120.h5', 'LSTM250.h5', 'LSTM60.h5']
members = load_all_models(members)
print('Loaded %d models' % len(members))

train_predictions = []
val_predictions = []
test_predictions = []

for model in members:
  train_predictions.append(model.predict([X_train], batch_size=1024))
  val_predictions.append(model.predict([X_val], batch_size=1024))
  test_predictions.append(model.predict([X_test], batch_size=1024))
  print('Predicted one thing')

>loaded /content/drive/MyDrive/twit/train/models/LSTM200.h5
>loaded /content/drive/MyDrive/twit/train/models/LSTM150.h5
>loaded /content/drive/MyDrive/twit/train/models/LSTM120.h5
>loaded /content/drive/MyDrive/twit/train/models/LSTM250.h5
>loaded /content/drive/MyDrive/twit/train/models/LSTM60.h5
Loaded 5 models
Predicted one thing
Predicted one thing
Predicted one thing
Predicted one thing
Predicted one thing


In [39]:
from keras.layers import BatchNormalization
def ensemble(predictions, targets, xval, yval):
    layer_size = len(predictions)
    inp = Input(shape=(layer_size,3))
    f0 = Flatten()(inp)
    d0 = Dropout(0.2)(f0)
    d0 = Dense(pow(layer_size, 4))(d0)
    d1 = Dropout(0.2)(d0)
    d1 = Dense(20 * layer_size)(d1)
    b = BatchNormalization()(d1)
    out = Dropout(0.2)(b)
    out = Dense(3, activation='sigmoid')(out)

    model = Model(inputs=inp, outputs=out)
    metrics = [
           keras.metrics.CategoricalAccuracy(),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall'),
           f1_metrics,
           #keras.metrics.TruePositives(name='tp'),
           #keras.metrics.FalsePositives(name='fp'),
           #keras.metrics.TrueNegatives(name='tn'),
           #keras.metrics.FalseNegatives(name='fn'),
  ]
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=metrics)
    model.summary()

    x = np.array(list(zip(*np.squeeze(predictions))))
    y = targets
    print(np.shape(x))
    print(np.shape(y))

    xval = np.array(list(zip(*np.squeeze(xval))))
    yval = yval
    
    history = model.fit(x=x, y=y, epochs=20, validation_data=(xval,yval), verbose=2)
    #display_model_history(history_flat_CNN) 
    #plot_metrics(history_flat_CNN)
    return model, history

In [40]:
y_train_cat = Y_train
y_val_cat = Y_val
y_test_cat = Y_test  
stack_model, history = ensemble(train_predictions, y_train_cat, val_predictions, y_val_cat)

stacked_test_predictions = np.array(list(zip(*np.squeeze(test_predictions))))
stacked_test_predictions = stack_model.predict(stacked_test_predictions, batch_size=1024)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 5, 3)]            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 15)                0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 15)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 625)               10000     
_________________________________________________________________
dropout_7 (Dropout)          (None, 625)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               62600     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400 

In [41]:
indiv = []
for model in members:
  indiv.append(f1_metrics(y_test_cat, model.predict(X_test)))

max(indiv)

<tf.Tensor: shape=(), dtype=float32, numpy=0.7172153>

In [43]:
stack_model.evaluate(np.array(list(zip(*np.squeeze(test_predictions)))),y_test_cat)

357/357 [==============================] - 1s 3ms/step - loss: 0.6881 - categorical_accuracy: 0.7197 - precision: 0.6716 - recall: 0.8112 - f1_metrics: 0.7366


[0.688101053237915,
 0.7196637988090515,
 0.6715952754020691,
 0.8112414479255676,
 0.7365813851356506]